# Master Budget Dashboard - Stellar Analytics Division

This comprehensive dashboard provides executive-level oversight of all operational category budgets within the Stellar Analytics Division (SAD). It enables:

- Cross-category budget analysis and comparison
- Division-wide spending patterns and trends
- Resource optimization recommendations
- Executive reporting and alerts
- Consolidated cost efficiency metrics

**Division**: Stellar Analytics Division (SAD)  
**Total Monthly Budget**: $72,000  
**Last Updated**: April 2025  
**Report Generated**: [Runtime]

In [0]:
# Global Parameters

# All operational categories and their monthly budgets
OPERATIONAL_CATEGORIES = {
    "DEEP_SPACE_TELEMETRY": 10000,
    "PROPULSION_ANALYTICS": 12000,
    "ORBITAL_MECHANICS": 8000,
    "MATERIALS_SCIENCE": 9000,
    "EXPLORATORY_MISSIONS": 15000,
    "NAVIGATION_SYSTEMS": 7000,
    "EXOPLANET_RESEARCH": 11000
}

# Total division budget
TOTAL_DIVISION_BUDGET = sum(OPERATIONAL_CATEGORIES.values())

# Date range for analysis
import datetime
current_date = datetime.datetime.now()
first_day_of_month = current_date.replace(day=1)
last_day_of_month = (first_day_of_month + datetime.timedelta(days=32)).replace(day=1) - datetime.timedelta(days=1)

DATE_RANGE = {
    "start": first_day_of_month.strftime("%Y-%m-%d"),
    "end": last_day_of_month.strftime("%Y-%m-%d")
}

# Alert thresholds
ALERT_THRESHOLDS = [75, 90, 100]

# Division configuration
DIVISION_NAME = "SAD"
COST_CENTER = "ND-Analytics"

# Report timestamp
REPORT_TIMESTAMP = current_date.strftime("%Y-%m-%d %H:%M:%S")

In [0]:
# Authentication and Imports

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, count, countDistinct
import requests
import json
import warnings
warnings.filterwarnings('ignore')

# Initialize Spark session
spark = SparkSession.builder.appName("MasterBudgetDashboard").getOrCreate()

# Configure visualization settings
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = [14, 8]

# Databricks API configuration (if needed for additional API calls)
DATABRICKS_INSTANCE = "your-instance.cloud.databricks.com"
TOKEN = "your-token"  # Replace with your token
API_BASE_URL = f"https://{DATABRICKS_INSTANCE}/api/2.0"

In [0]:
# Data Aggregation Functions

def aggregate_all_budgets():
    """Aggregate budget data across all operational categories."""
    category_list = list(OPERATIONAL_CATEGORIES.keys())
    category_str = ", ".join([f"'{cat}'" for cat in category_list])
    
    query = f"""
    SELECT 
        tags.OperationalCategory as category,
        SUM(usage_quantity * list_price) as total_cost,
        COUNT(DISTINCT usage_metadata.cluster_id) as cluster_count,
        COUNT(DISTINCT usage_metadata.warehouse_id) as warehouse_count,
        COUNT(DISTINCT tags.Owner) as user_count,
        COUNT(DISTINCT sku_name) as resource_types,
        MIN(usage_date) as first_usage,
        MAX(usage_date) as last_usage
    FROM system.billing.usage
    WHERE tags.OperationalCategory IN ({category_str})
    AND usage_date BETWEEN '{DATE_RANGE['start']}' AND '{DATE_RANGE['end']}'
    GROUP BY tags.OperationalCategory
    ORDER BY total_cost DESC
    """
    return spark.sql(query)

def get_daily_trends():
    """Get daily spending trends for all categories."""
    category_list = list(OPERATIONAL_CATEGORIES.keys())
    category_str = ", ".join([f"'{cat}'" for cat in category_list])
    
    query = f"""
    SELECT 
        usage_date,
        tags.OperationalCategory as category,
        SUM(usage_quantity * list_price) as daily_cost
    FROM system.billing.usage
    WHERE tags.OperationalCategory IN ({category_str})
    AND usage_date BETWEEN '{DATE_RANGE['start']}' AND '{DATE_RANGE['end']}'
    GROUP BY usage_date, tags.OperationalCategory
    ORDER BY usage_date, category
    """
    return spark.sql(query)

def get_resource_distribution():
    """Get resource usage distribution across categories."""
    category_list = list(OPERATIONAL_CATEGORIES.keys())
    category_str = ", ".join([f"'{cat}'" for cat in category_list])
    
    query = f"""
    SELECT 
        tags.OperationalCategory as category,
        sku_name,
        SUM(usage_quantity * list_price) as resource_cost,
        SUM(usage_quantity) as usage_amount
    FROM system.billing.usage
    WHERE tags.OperationalCategory IN ({category_str})
    AND usage_date BETWEEN '{DATE_RANGE['start']}' AND '{DATE_RANGE['end']}'
    GROUP BY tags.OperationalCategory, sku_name
    ORDER BY category, resource_cost DESC
    """
    return spark.sql(query)

def get_user_activity():
    """Get user activity across categories."""
    category_list = list(OPERATIONAL_CATEGORIES.keys())
    category_str = ", ".join([f"'{cat}'" for cat in category_list])
    
    query = f"""
    SELECT 
        tags.OperationalCategory as category,
        tags.Owner as user_email,
        SUM(usage_quantity * list_price) as user_cost,
        COUNT(DISTINCT usage_date) as active_days
    FROM system.billing.usage
    WHERE tags.OperationalCategory IN ({category_str})
    AND tags.Owner IS NOT NULL
    AND usage_date BETWEEN '{DATE_RANGE['start']}' AND '{DATE_RANGE['end']}'
    GROUP BY tags.OperationalCategory, tags.Owner
    ORDER BY user_cost DESC
    """
    return spark.sql(query)

In [0]:
# Execute data collection
print(f"Collecting data for all operational categories from {DATE_RANGE['start']} to {DATE_RANGE['end']}...")

# Aggregate data
category_summary = aggregate_all_budgets().toPandas()
daily_trends = get_daily_trends().toPandas()
resource_distribution = get_resource_distribution().toPandas()
user_activity = get_user_activity().toPandas()

# Calculate key metrics
total_division_spend = category_summary['total_cost'].sum() if not category_summary.empty else 0
division_utilization = (total_division_spend / TOTAL_DIVISION_BUDGET) * 100

print(f"\nDivision-wide Summary:")
print(f"Total Monthly Budget: ${TOTAL_DIVISION_BUDGET:,.2f}")
print(f"Total Spent: ${total_division_spend:,.2f}")
print(f"Budget Utilization: {division_utilization:.1f}%")
print(f"Active Categories: {len(category_summary)}")

In [0]:
# Executive Summary Visualization

def generate_executive_summary():
    """Create executive summary dashboard with key metrics."""
    # Create subplots for executive KPIs
    fig = make_subplots(
        rows=1, cols=3,
        specs=[[{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}]],
        subplot_titles=["Division Budget Utilization", "Categories in Alert", "Projected Overspend"]
    )
    
    # Budget utilization gauge
    fig.add_trace(
        go.Indicator(
            mode="gauge+number",
            value=division_utilization,
            number={'suffix': "%"},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': "darkblue"},
                'steps': [
                    {'range': [0, 75], 'color': "lightgreen"},
                    {'range': [75, 90], 'color': "yellow"},
                    {'range': [90, 100], 'color': "red"}
                ]
            }
        ),
        row=1, col=1
    )
    
    # Categories in alert indicator
    if not category_summary.empty:
        categories_in_alert = 0
        for _, row in category_summary.iterrows():
            budget = OPERATIONAL_CATEGORIES[row['category']]
            utilization = (row['total_cost'] / budget) * 100
            if utilization >= 75:
                categories_in_alert += 1
    else:
        categories_in_alert = 0
    
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=categories_in_alert,
            number={'font': {'size': 48, 'color': "red" if categories_in_alert > 2 else "orange" if categories_in_alert > 0 else "green"}}
        ),
        row=1, col=2
    )
    
    # Calculate projected overspend
    days_elapsed = (current_date - first_day_of_month).days + 1
    days_in_month = (last_day_of_month - first_day_of_month).days + 1
    if days_elapsed > 0:
        daily_rate = total_division_spend / days_elapsed
        projected_total = total_division_spend + (daily_rate * (days_in_month - days_elapsed))
        projected_overspend = max(0, projected_total - TOTAL_DIVISION_BUDGET)
    else:
        projected_overspend = 0
    
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=projected_overspend,
            number={'prefix': "$", 'valueformat': ",.2f", 'font': {'color': "red" if projected_overspend > 0 else "green"}}
        ),
        row=1, col=3
    )
    
    fig.update_layout(height=300, title_text="Executive Budget Summary")
    fig.show()

generate_executive_summary()

In [0]:
# Category Distribution Pie Chart

def create_category_distribution():
    """Create pie chart showing budget distribution by category."""
    if not category_summary.empty:
        # Add budget info to category summary
        category_summary['budget'] = category_summary['category'].map(OPERATIONAL_CATEGORIES)
        category_summary['utilization'] = (category_summary['total_cost'] / category_summary['budget']) * 100
        
        fig = go.Figure(data=[go.Pie(
            labels=category_summary['category'],
            values=category_summary['total_cost'],
            hole=.4,
            textinfo='label+percent',
            textposition='outside',
            marker=dict(colors=px.colors.qualitative.Set3),
            customdata=category_summary[['budget', 'utilization']],
            hovertemplate="Category: %{label}<br>Spent: $%{value:,.2f}<br>Budget: $%{customdata[0]:,.2f}<br>Utilization: %{customdata[1]:.1f}%<extra></extra>"
        )])
        
        fig.update_layout(
            title="Division Budget Distribution by Category",
            height=600,
            annotations=[dict(text=f'Total: ${total_division_spend:,.0f}', x=0.5, y=0.5, font_size=20, showarrow=False)]
        )
        
        fig.show()
    else:
        print("No data available for category distribution.")

create_category_distribution()

In [0]:
# Historical Spending Stacked Bar Chart

def create_historical_spending():
    """Create stacked bar chart of historical spending by category."""
    if not daily_trends.empty:
        # Pivot the data for stacked chart
        daily_pivot = daily_trends.pivot(index='usage_date', columns='category', values='daily_cost').fillna(0)
        
        fig = go.Figure()
        
        for category in daily_pivot.columns:
            fig.add_trace(go.Bar(
                name=category,
                x=daily_pivot.index,
                y=daily_pivot[category],
                hovertemplate="%{x}<br>%{y:$,.2f}<extra></extra>"
            ))
        
        fig.update_layout(
            barmode='stack',
            title='Daily Spending by Category',
            xaxis_title='Date',
            yaxis_title='Cost (USD)',
            height=600,
            legend_title='Category',
            hovermode='x unified'
        )
        
        fig.show()
    else:
        print("No daily trend data available.")

create_historical_spending()

In [0]:
# Resource Utilization Heatmap

def create_resource_heatmap():
    """Create heatmap of resource utilization by category."""
    if not resource_distribution.empty:
        # Pivot data for heatmap
        resource_pivot = resource_distribution.pivot(index='category', columns='sku_name', values='resource_cost').fillna(0)
        
        # Create heatmap
        fig = px.imshow(
            resource_pivot,
            labels=dict(x="Resource Type", y="Category", color="Cost (USD)"),
            title="Resource Cost Heatmap by Category",
            aspect="auto",
            color_continuous_scale="Viridis"
        )
        
        fig.update_layout(
            height=800,
            xaxis_tickangle=-45
        )
        
        fig.show()
    else:
        print("No resource distribution data available.")

create_resource_heatmap()

In [0]:
# Alert Status Dashboard

def create_alert_dashboard():
    """Create visual dashboard of alert status for all categories."""
    if not category_summary.empty:
        alerts_data = []
        
        for _, row in category_summary.iterrows():
            category = row['category']
            budget = OPERATIONAL_CATEGORIES[category]
            cost = row['total_cost']
            utilization = (cost / budget) * 100
            
            status = "CRITICAL" if utilization >= 100 else "WARNING" if utilization >= 90 else "CAUTION" if utilization >= 75 else "NORMAL"
            color = "red" if status == "CRITICAL" else "orange" if status == "WARNING" else "yellow" if status == "CAUTION" else "green"
            
            alerts_data.append({
                'category': category,
                'budget': budget,
                'spent': cost,
                'utilization': utilization,
                'status': status,
                'color': color
            })
        
        alerts_df = pd.DataFrame(alerts_data)
        
        # Create status indicator chart
        fig = go.Figure()
        
        for i, row in alerts_df.iterrows():
            fig.add_trace(go.Bar(
                y=[row['category']],
                x=[row['utilization']],
                name=row['category'],
                orientation='h',
                marker_color=row['color'],
                text=f"{row['utilization']:.1f}% ({row['status']})",
                textposition='inside',
                hovertemplate="Category: %{y}<br>Utilization: %{x:.1f}%<br>Status: " + row['status'] + "<extra></extra>"
            ))
        
        # Add threshold lines
        for threshold in ALERT_THRESHOLDS:
            fig.add_vline(x=threshold, line_dash="dash", line_color="black", annotation_text=f"{threshold}%")
        
        fig.update_layout(
            title='Category Alert Status Dashboard',
            xaxis_title='Budget Utilization (%)',
            yaxis_title='Category',
            height=600,
            showlegend=False,
            xaxis_range=[0, max(120, alerts_df['utilization'].max() + 10)]
        )
        
        fig.show()
        
        # Display detailed alert table
        display(alerts_df.style.apply(lambda row: [f'background-color: {row["color"]}' for _ in row], axis=1))
    else:
        print("No category data available for alert dashboard.")

create_alert_dashboard()

In [0]:
# Comparative Analysis

def create_comparative_analysis():
    """Create comparative analysis charts for categories."""
    if not category_summary.empty:
        # Add budget and utilization data
        category_summary['budget'] = category_summary['category'].map(OPERATIONAL_CATEGORIES)
        category_summary['utilization'] = (category_summary['total_cost'] / category_summary['budget']) * 100
        category_summary['efficiency'] = category_summary['total_cost'] / category_summary['user_count'].replace(0, 1)
        
        # Create comparative metrics chart
        fig = make_subplots(rows=2, cols=2, 
                           subplot_titles=("Budget vs Actual Spending", 
                                         "Resource Efficiency", 
                                         "User Activity", 
                                         "Category Efficiency Score"))
        
        # Budget vs Actual
        fig.add_trace(
            go.Bar(name='Budget', x=category_summary['category'], y=category_summary['budget'],
                  marker_color='lightblue'),
            row=1, col=1
        )
        fig.add_trace(
            go.Bar(name='Actual', x=category_summary['category'], y=category_summary['total_cost'],
                  marker_color='darkblue'),
            row=1, col=1
        )
        
        # Resource Efficiency (Cost per resource type)
        if 'resource_types' in category_summary.columns:
            fig.add_trace(
                go.Bar(x=category_summary['category'], 
                      y=category_summary['total_cost'] / category_summary['resource_types'].replace(0, 1),
                      marker_color='green',
                      name='Cost per Resource Type'),
                row=1, col=2
            )
        
        # User Activity
        fig.add_trace(
            go.Bar(x=category_summary['category'], 
                  y=category_summary['user_count'],
                  marker_color='purple',
                  name='Active Users'),
            row=2, col=1
        )
        
        # Efficiency Score (composite of various metrics)
        category_summary['efficiency_score'] = (1 / (category_summary['utilization'] / 100)) * \
                                             (category_summary['user_count'] / category_summary['user_count'].max()) * \
                                             (1 / (category_summary['efficiency'] / category_summary['efficiency'].max()))
        
        fig.add_trace(
            go.Bar(x=category_summary['category'], 
                  y=category_summary['efficiency_score'],
                  marker_color='orange',
                  name='Efficiency Score'),
            row=2, col=2
        )
        
        fig.update_layout(height=800, showlegend=False, title_text="Comparative Category Analysis")
        fig.update_xaxes(tickangle=45)
        fig.show()

create_comparative_analysis()

In [0]:
# Optimization Recommendations (continued)

                    'issue': f"{utilization:.1f}% utilized - Exceeds budget by ${cost - budget:.2f}",
                    'recommendation': "Implement immediate cost controls and review resource allocation",
                    'priority': 'HIGH'
                })
            elif utilization > 90:
                recommendations.append({
                    'category': category,
                    'type': 'Budget Warning',
                    'issue': f"{utilization:.1f}% utilized - Approaching budget limit",
                    'recommendation': "Review upcoming workloads and consider resource optimization",
                    'priority': 'MEDIUM'
                })
            
            # Check user efficiency
            if row['user_count'] > 0:
                cost_per_user = cost / row['user_count']
                avg_cost_per_user = total_division_spend / category_summary['user_count'].sum()
                
                if cost_per_user > avg_cost_per_user * 1.5:
                    recommendations.append({
                        'category': category,
                        'type': 'User Efficiency',
                        'issue': f"High cost per user (${cost_per_user:.2f} vs avg ${avg_cost_per_user:.2f})",
                        'recommendation': "Audit user permissions and optimize resource sharing",
                        'priority': 'MEDIUM'
                    })
    
    # Resource utilization recommendations
    if not resource_distribution.empty:
        # Group by resource type across categories
        resource_totals = resource_distribution.groupby('sku_name')['resource_cost'].sum().sort_values(ascending=False)
        
        if not resource_totals.empty:
            top_resources = resource_totals.head(3)
            for resource, cost in top_resources.items():
                if cost > total_division_spend * 0.2:  # Resource using >20% of budget
                    recommendations.append({
                        'category': 'Division-wide',
                        'type': 'Resource Concentration',
                        'issue': f"{resource} accounts for {cost/total_division_spend*100:.1f}% of total spend",
                        'recommendation': "Evaluate alternatives or negotiate better rates for high-cost resources",
                        'priority': 'MEDIUM'
                    })
    
    # Trend-based recommendations
    days_elapsed = (current_date - first_day_of_month).days + 1
    days_in_month = (last_day_of_month - first_day_of_month).days + 1
    if days_elapsed > 7:  # Need at least a week of data
        projected_total = (total_division_spend / days_elapsed) * days_in_month
        
        if projected_total > TOTAL_DIVISION_BUDGET * 1.1:
            recommendations.append({
                'category': 'Division-wide',
                'type': 'Spending Trend',
                'issue': f"Projected month-end spend: ${projected_total:.2f} ({projected_total/TOTAL_DIVISION_BUDGET*100:.1f}% of budget)",
                'recommendation': "Implement division-wide cost reduction measures immediately",
                'priority': 'HIGH'
            })
    
    # Display recommendations
    if recommendations:
        rec_df = pd.DataFrame(recommendations)
        
        # Color coding for priorities
        def highlight_priority(row):
            if row['priority'] == 'HIGH':
                return ['background-color: #ffcccb'] * len(row)
            elif row['priority'] == 'MEDIUM':
                return ['background-color: #ffffcc'] * len(row)
            else:
                return [''] * len(row)
        
        print("\n=== OPTIMIZATION RECOMMENDATIONS ===")
        display(rec_df.style.apply(highlight_priority, axis=1))
    else:
        print("\n=== OPTIMIZATION RECOMMENDATIONS ===")
        print("No specific optimization recommendations at this time.")
    
    return recommendations

# Generate and display recommendations
recommendations = generate_optimization_recommendations()

In [0]:
# Export Leadership Report

def export_leadership_report():
    """Generate comprehensive leadership report in PDF/HTML format."""
    # Create comprehensive summary DataFrame
    report_data = []
    
    if not category_summary.empty:
        for _, row in category_summary.iterrows():
            category = row['category']
            budget = OPERATIONAL_CATEGORIES[category]
            cost = row['total_cost']
            utilization = (cost / budget) * 100
            
            report_data.append({
                'Category': category.replace('_', ' ').title(),
                'Budget': f"${budget:,.2f}",
                'Spent': f"${cost:,.2f}",
                'Remaining': f"${budget - cost:,.2f}",
                'Utilization': f"{utilization:.1f}%",
                'Active Users': row['user_count'],
                'Resources Used': row.get('resource_types', 0),
                'Clusters': row.get('cluster_count', 0),
                'Warehouses': row.get('warehouse_count', 0),
                'Status': "CRITICAL" if utilization >= 100 else "WARNING" if utilization >= 90 else "CAUTION" if utilization >= 75 else "NORMAL"
            })
    
    report_df = pd.DataFrame(report_data)
    
    # Create summary statistics
    summary_stats = {
        'Report Date': REPORT_TIMESTAMP,
        'Period': f"{first_day_of_month.strftime('%B %Y')}",
        'Total Division Budget': f"${TOTAL_DIVISION_BUDGET:,.2f}",
        'Total Spent': f"${total_division_spend:,.2f}",
        'Overall Utilization': f"{division_utilization:.1f}%",
        'Categories in Alert': sum(1 for _, row in category_summary.iterrows() if (row['total_cost'] / OPERATIONAL_CATEGORIES[row['category']]) >= 75),
        'Total Active Users': category_summary['user_count'].sum() if not category_summary.empty else 0,
        'Total Resources Used': category_summary['resource_types'].sum() if not category_summary.empty and 'resource_types' in category_summary.columns else 0
    }
    
    # Export to HTML
    html_content = f"""
    <html>
    <head>
        <title>Stellar Analytics Division Budget Report - {first_day_of_month.strftime('%B %Y')}</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            h1 {{ color: #333; }}
            h2 {{ color: #666; }}
            table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
            th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
            th {{ background-color: #f2f2f2; }}
            .critical {{ background-color: #ffcccb; }}
            .warning {{ background-color: #ffffcc; }}
            .caution {{ background-color: #fffacd; }}
            .normal {{ background-color: #90ee90; }}
            .summary {{ background-color: #f8f9fa; padding: 20px; border-radius: 5px; margin-bottom: 30px; }}
        </style>
    </head>
    <body>
        <h1>Stellar Analytics Division Budget Report</h1>
        <h2>{first_day_of_month.strftime('%B %Y')}</h2>
        
        <div class="summary">
            <h3>Executive Summary</h3>
            <table>
                {''.join([f'<tr><td><strong>{k}</strong></td><td>{v}</td></tr>' for k, v in summary_stats.items()])}
            </table>
        </div>
        
        <h3>Category Details</h3>
        {report_df.to_html(index=False, classes='table table-striped', escape=False)}
        
        <h3>Optimization Recommendations</h3>
        {pd.DataFrame(recommendations).to_html(index=False, classes='table table-striped') if recommendations else '<p>No specific recommendations at this time.</p>'}
        
        <p><small>Generated on {REPORT_TIMESTAMP}</small></p>
    </body>
    </html>
    """
    
    # Save files
    html_filename = f"stellar_analytics_budget_report_{current_date.strftime('%Y%m%d')}.html"
    csv_filename = f"stellar_analytics_budget_summary_{current_date.strftime('%Y%m%d')}.csv"
    
    with open(html_filename, 'w') as f:
        f.write(html_content)
    
    report_df.to_csv(csv_filename, index=False)
    
    print(f"\n=== REPORTS GENERATED ===")
    print(f"HTML Report: {html_filename}")
    print(f"CSV Summary: {csv_filename}")
    
    # Display summary in notebook
    print("\n=== BUDGET SUMMARY ===")
    display(pd.DataFrame([summary_stats]))
    
    print("\n=== CATEGORY DETAILS ===")
    display(report_df.style.apply(lambda row: ['background-color: #ffcccb' if row['Status'] == 'CRITICAL' else 
                                             'background-color: #ffffcc' if row['Status'] == 'WARNING' else 
                                             'background-color: #fffacd' if row['Status'] == 'CAUTION' else 
                                             'background-color: #90ee90' for _ in row], axis=1))

# Export the leadership report
export_leadership_report()

## Master Budget Dashboard - Summary and Next Steps

This comprehensive dashboard provides executive-level oversight of all Stellar Analytics Division operational budgets. Key features include:

### Dashboard Components
1. **Executive Summary**: High-level KPIs including budget utilization and alert status
2. **Category Distribution**: Pie chart showing spend allocation across operational areas
3. **Historical Trends**: Stacked bar chart showing daily spending patterns
4. **Resource Heatmap**: Visualization of resource usage by category
5. **Alert Dashboard**: Visual status indicators for budget thresholds
6. **Comparative Analysis**: Multi-metric comparison across categories
7. **Optimization Recommendations**: Automated insights and cost-saving opportunities
8. **Leadership Reports**: Exportable HTML and CSV summaries

### Action Items
1. Review categories in alert status immediately
2. Implement recommended optimization strategies
3. Schedule monthly reviews with category owners
4. Monitor projected overspend and take corrective action
5. Share exported reports with division leadership

### Automated Monitoring
- This dashboard should be scheduled to run daily
- Alerts will be automatically sent when thresholds are reached
- Weekly summary reports should be distributed to leadership

For support or questions: 